In [2]:
from langchain.llms import HuggingFacePipeline
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, AutoModelForSeq2SeqLM,T5ForConditionalGeneration

c:\Users\Sayantika\anaconda3\envs\myenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:

if torch.cuda.is_available():
    print("CUDA is available on your system.")
else:
    print("CUDA is not available on your system.")


CUDA is available on your system.


Running Model From Hugging Face Locally

In [2]:
model_id = 'MBZUAI/LaMini-Flan-T5-248M'
tokenizer = AutoTokenizer.from_pretrained(model_id,device_map='auto')
model = AutoModelForSeq2SeqLM.from_pretrained(model_id,device_map='auto')

c:\Users\Sayantika\anaconda3\envs\myenv\lib\site-packages\huggingface_hub\file_download.py:137: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Sayantika\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [4]:
pipeline = pipeline("summarization", model=model,tokenizer=tokenizer, max_length=600)

In [5]:
llm = HuggingFacePipeline(pipeline=pipeline,model_kwargs={"temperature":5})

In [5]:

from langchain.chains.summarize import load_summarize_chain
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [7]:
loader=PyPDFLoader(r".\data\2023_GPT4All_Technical_Report.pdf")

In [8]:
docs=loader.load()

In [10]:
docs[0]

Document(page_content='GPT4All: Training an Assistant-style Chatbot with Large Scale Data\nDistillation from GPT-3.5-Turbo\nYuvanesh Anand\nyuvanesh@nomic.aiZach Nussbaum\nzanussbaum@gmail.com\nBrandon Duderstadt\nbrandon@nomic.aiBenjamin Schmidt\nben@nomic.aiAndriy Mulyar\nandriy@nomic.ai\nAbstract\nThis preliminary technical report describes the\ndevelopment of GPT4All, a chatbot trained\nover a massive curated corpus of assistant in-\nteractions including word problems, story de-\nscriptions, multi-turn dialogue, and code. We\nopenly release the collected data, data cura-\ntion procedure, training code, and final model\nweights to promote open research and repro-\nducibility. Additionally, we release quantized\n4-bit versions of the model allowing virtually\nanyone to run the model on CPU.\n1 Data Collection and Curation\nWe collected roughly one million prompt-\nresponse pairs using the GPT-3.5-Turbo OpenAI\nAPI between March 20, 2023 and March 26th,\n2023. To do this, we first gat

In [11]:
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

In [12]:
texts=text_splitter.split_documents(docs)

In [13]:
len(texts)

8

In [16]:
chain=load_summarize_chain(llm, chain_type='map_reduce')

In [17]:
chain.run(texts)

Your max_length is set to 600, but your input_length is only 298. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=149)


Your max_length is set to 600, but your input_length is only 273. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=136)
Your max_length is set to 600, but your input_length is only 281. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=140)
Your max_length is set to 600, but your input_length is only 153. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=76)
Your max_length is set to 600, but your input_length is only 267. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=13

'GPT4All is a chatbot trained over a large corpus of assistant in-teractions, including word problems, story de-scriptions, multi-turn dialogue, and code. The data collection and cura-tion procedure, training code, and final model weights were collected using the GPT-3.5-Turbo OpenAI API between March 20, 2023 and March 26th, 2023. Quantized 4-bit versions of the model were released. The datasets included the chip2 subset of LAION OIG, Stackoverflow Questions, and instruction-tuning with a sub-sample of Big-science/P3. Data preparation and curation were based on commentary in the Stanford Alpaca project. Data was loaded into Atlas for data curation and cleaning. The total number of examples reduced to 806,199 high-quality prompt-generation pairs. The model training process involved cleaning and fine-ting models from an in-stance of LLaMA 7B and LoRA on 437,605 post-processed examples for four epochs. The final training data was curated to ensure a diverse distribution of prompt topics 